## SNO+ Scattering Pseudo-MC
A simple test using known results to determine the viability of nuclear recoil
explaining the odd behaviour at 2.2 MeV.

In [37]:
# importing the usual suspects
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

In [51]:
# now we read in the stopping power data from ASTAR
stoppingpowers = pd.read_csv(Path("stoppingpowers/paraffin.csv"))
# rename cols to make them easier to reference
stoppingpowers.columns = ["KE", "electron", "nuclear", "total"]
# reindex the dataframe by kinetic energy to make lookups easier
#stoppingpowers.set_index("KE", inplace=True)
print(stoppingpowers)

            KE  electron    nuclear  total
0       0.0010     194.0  289.00000  483.0
1       0.0015     233.0  260.00000  493.0
2       0.0020     266.0  238.00000  503.0
3       0.0025     294.0  219.00000  514.0
4       0.0030     320.0  204.00000  524.0
..         ...       ...        ...    ...
116   800.0000      19.1    0.00655   19.1
117   850.0000      18.4    0.00617   18.4
118   900.0000      17.7    0.00582   17.7
119   950.0000      17.1    0.00552   17.2
120  1000.0000      16.6    0.00524   16.6

[121 rows x 4 columns]


So now we have the stopping power data available to us and formatted nicely.

**NB**: the initial revision here is without scattering to get a grip on things

The setup is that we have some sort of particle with a given energy E_0 (i'm not sure exactly what yet -
need to figure that out)
The general process goes like this:
- Look up stopping power in the table for the energy of our event
- Find dE (**NOTE**: need to figure out what dx will be here first - see
  wikipedia page for stopping power)
- Add dE to the bin for an event of that energy
- Repeat this, as if we had an energy of E_j = E_0 - dE
- Terminate once E_j is less than some given epsilon (generally close to zero -
  but we will never reach zero exactly)

Implementation notes: Can use recursion, but that is setting myself up for a
nightmare so let's keep things iterative

Also - how to deal with events which aren't exactly a given KE in the table.
Data is nonlinear but describes the derivative of energy w.r.t. position - the
function "looks like" it is C-infinity (ie. continuous derivatives) and differentiability
implies linearity in a neighbourhood so a linear interpolation should be a very
accurate estimation 

In [39]:
# Let's set up the bins for events here
# I don't know of any prepackaged solution for this so let's write a structure
# to hold binned data for us

# this kind of seems unnecessary to me, but also I would be implementing the
# exact same logic in a messier form otherwise so I digress
class CountBin:
    def __init__(self, low, high):
        self._low = low
        self._high = high
        self._count = 0
    
    # setting properties for the bin count
    @property
    def count(self):
        return self._count

    @count.setter
    def count(self, new_count):
        self._count = new_count

    # increment and decrement convenience functions
    def increment(self):
        self.count = self._count + 1       

    def decrement(self):
        self.count = self._count - 1

    # properties for bin bounds
    @property
    def low_range(self):
        return self._low

    @property
    def high_range(self):
        return self._high

    def __str__(self):
        return f"CountBin(low={self.low_range}, high={self.high_range}, count={self.count})"
    

class BinnedCountData:
    def __init__(self, num_bins, d):
        low = min(d)
        high = max(d)

        self.binEdges = np.linspace(low, high, num_bins+1)

        self.bins = []

        # start count at one intentionally here
        i = 1
        while i < len(self.binEdges):
            self.bins.append(CountBin(self.binEdges[i-1], self.binEdges[i]))
            i += 1

    def get_bin_index(self, v):
        i = 0
        while i < len(self.binEdges) - 1:
            if v < self.binEdges[i+1]:
                return i
            i += 1

    # add a value to the correct count bin
    def count_value(self, v):
        # TODO is there a faster algorithm for this?
        self.bins[self.get_bin_index(v)].increment()
    # count an entire list
    def count_list(self, l):
        for i in l:
            self.count_value(i)

    @property
    def counts(self):
        return sum([i.count for i in self.bins])

    def count_by_value(self, v):
        # get count of a bin some value is in
        return self.bins[self.get_bin_index(v)].count

Now we do the linear interpolation of the ASTAR data
Although the data is nonlinear, this is still decently accurate since it is a differentiable function 

In [ ]:
# now to do the linear interpolation for the ASTAR dataset
for k in stoppingpowers.index:
    print(stoppingpowers["KE"][k])

def stp_interp(e, stp=stoppingpowers):
    # NOTE this assumes that the stopping powers are sorted
    # we get them this way from ASTAR so it's not an issue, but we can fix that if need be
    for k in stoppingpowers.index:
        if stoppingpowers["KE"][k] <= e:
            ke_low = stoppingpowers["KE"][k]
            sp_low = stoppingpowers["total"][k]
            ke_high = stoppingpowers["KE"][k+1]
            sp_high = stoppingpowers["total"][k+1]
            break

    return (sp_high - sp_low)/(ke_high - ke_low) * e + sp_low

Now, let's do the simulation. Let's do one without any nuclear scattering first to get things working, and then we'll figure out the cross-sections and modify things later.

In [ ]:
stepsize = 0.1 # this is our dx
e_0 = 4 # initial energy in MeV